In [1]:
import pandas as pd
from pympler.asizeof import asizeof

In [2]:
def get_csv():
    return pd.read_csv('10000 Sales Records.csv')

In [3]:
df = get_csv()

In [4]:
df = df.set_index("Order Date").sort_index()  # Setting the index to the Order Date

In [5]:
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
Order Date,,,,,,,,,,,,,
1/1/2010,Sub-Saharan Africa,Central African Republic,Vegetables,Online,H,506209075,2/4/2010,7369,154.06,90.93,1135268.14,670063.17,465204.97
1/1/2010,Asia,China,Cereal,Online,C,863776719,2/10/2010,9581,205.70,117.11,1970811.70,1122030.91,848780.79
1/1/2011,Europe,Spain,Household,Online,C,125338399,1/27/2011,9272,668.27,502.54,6196199.44,4659550.88,1536648.56
1/1/2011,Europe,United Kingdom,Snacks,Online,H,983697640,2/10/2011,6847,152.58,97.44,1044715.26,667171.68,377543.58
1/1/2011,Europe,Greece,Clothes,Offline,H,992002189,1/13/2011,4347,109.28,35.84,475040.16,155796.48,319243.68


In [6]:
df.loc["1/1/2011"]

,Region,Country,Item Type,Sales Channel,Order Priority,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
Order Date,,,,,,,,,,,,,
1/1/2011,Europe,Spain,Household,Online,C,125338399,1/27/2011,9272,668.27,502.54,6196199.44,4659550.88,1536648.56
1/1/2011,Europe,United Kingdom,Snacks,Online,H,983697640,2/10/2011,6847,152.58,97.44,1044715.26,667171.68,377543.58
1/1/2011,Europe,Greece,Clothes,Offline,H,992002189,1/13/2011,4347,109.28,35.84,475040.16,155796.48,319243.68


In [7]:
# Cool. Let's do something more interesting
df = get_csv().set_index(["Country", "Item Type"]).sort_index()

In [8]:
str(df.index)[:100] + '\n...'  # Shows that this is a MultiIndex, since we have multiple items on each index

"MultiIndex(levels=[['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda '\n..."

In [9]:
df.loc["China", "Snacks"]  # Now we can retrieve every order of a certain item type from a certain country

Region Sales Channel Order Priority Order Date   Order ID  \
Country Item Type                                                             
China   Snacks      Asia        Online              C  10/3/2010  143867099   
        Snacks      Asia       Offline              C  4/30/2015  989680039   
        Snacks      Asia       Offline              L  10/1/2010  821808946   

                    Ship Date  Units Sold  Unit Price  Unit Cost  \
Country Item Type                                                  
China   Snacks     10/12/2010        2326      152.58      97.44   
        Snacks      5/31/2015        7171      152.58      97.44   
        Snacks     10/10/2010         384      152.58      97.44   

                   Total Revenue  Total Cost  Total Profit  
Country Item Type                                           
China   Snacks         354901.08   226645.44     128255.64  
        Snacks        1094151.18   698742.24     395408.94  
        Snacks          58590.72    37416.96      21173.76

In [10]:
get_csv().groupby(["Country", "Item Type"]).get_group(("China", "Snacks"))
# Performs the same operation as before, but the index remains intact

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
1817,Asia,China,Snacks,Online,C,10/3/2010,143867099,10/12/2010,2326,152.58,97.44,354901.08,226645.44,128255.64
3568,Asia,China,Snacks,Offline,C,4/30/2015,989680039,5/31/2015,7171,152.58,97.44,1094151.18,698742.24,395408.94
5616,Asia,China,Snacks,Offline,L,10/1/2010,821808946,10/10/2010,384,152.58,97.44,58590.72,37416.96,21173.76


In [11]:
from timeit import timeit
setup = "import pandas as pd\ndf = pd.read_csv('10000 Sales Records.csv')"

timeit('df.groupby(["Country", "Item Type"]).get_group(("China", "Snacks"))',
       setup=setup,
       number=1000)

3.572836612

In [12]:
timeit('df.set_index(["Country", "Item Type"]).sort_index().loc["China", "Snacks"]',
      setup=setup,
      number=1000)

13.625581333

In [13]:
# On my machine, the grouping example was more than 3 times faster
# Regardless, the grouping example seems a lot simpler and easier to follow along

In [14]:
df = get_csv()

In [15]:
df["Item Type"] = df["Item Type"].astype('category')
df["Region"] = df["Region"].astype('category')
df["Country"] = df["Country"].astype('category')
df["Order Priority"] = df["Order Priority"].astype('category')
df["Sales Channel"] = df["Sales Channel"].astype('category')

In [16]:
df_size = asizeof(df)
original_df_size = asizeof(get_csv())
original_df_size - df_size

6658896

In [17]:
# On my machine, the new DataFrame is ~6mb smaller than the original (original was about 10mb)
# So it seems categories are more memory efficient in the proper circumstances, just like the docs said
# Let's see how our grouping benchmark stacks up now

In [18]:
setup += """
df["Item Type"] = df["Item Type"].astype('category')
df["Region"] = df["Region"].astype('category')
df["Country"] = df["Country"].astype('category')
df["Order Priority"] = df["Order Priority"].astype('category')
df["Sales Channel"] = df["Sales Channel"].astype('category')"""

In [19]:
timeit('df.groupby(["Country", "Item Type"]).get_group(("China", "Snacks"))',
       setup=setup,
       number=1000)

3.0182341390000005

In [21]:
# Down from 3.7 seconds to 2.8 on my machine. Not bad for such an easy optimization!

In [24]:
type(df.loc[0, "Region"])
# And it seems it's only a category from within Pandas
# So we don't have to deal with the conversion ourselves, neat

str